In [10]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters

In [11]:
N = 500 #agents number
m = 3 #memory length
s = 4 #number of strategies held by each agent
T = 100 #memory of sore function
r = 4 #confidence level( used in CGMG)
L = 0 #size of increacement judged by each agent
step = 500 #total time steps
k0 = 2 #fraction of lambda and N_active

# Make Necessary Functions
### Strategy books: FSS and RSS.
### X[t] and mu[t] and S[t] D[t] denpend on the t-1 step and some other parameters.

In [12]:
#FSS:
def FSS(m):
    P = 2**m
    a = [1,-1]
    sb = np.zeros([2**P,P],dtype=int)
    def generate_permutations(length, current_permutation=[]):
        if length == 0:
            yield current_permutation
        else:
            for value in a:
                for permutation in generate_permutations(length - 1, current_permutation + [value]):
                    yield permutation

    all_permutations = np.array(list(generate_permutations(P)))
    return all_permutations

print(FSS(m))
        

[[ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1 -1]
 [ 1  1  1 ...  1 -1  1]
 ...
 [-1 -1 -1 ... -1  1 -1]
 [-1 -1 -1 ... -1 -1  1]
 [-1 -1 -1 ... -1 -1 -1]]


In [13]:
#RSS
'''
def RSS(m):
    a = np.array([[1,1,1,1,1,1,1,1],
                  [-1,-1,-1,-1,1,1,1,1],
                  [-1,-1,1,1,-1,-1,1,1],
                  [1,1,-1,-1,-1,-1,1,1],
                  [1,-1,-1,1,1,-1,-1,1],
                  [1,-1,-1,1,-1,1,1,-1],
                  [-1,1,1,-1,-1,1,1,-1],
                  [-1,1,-1,1,-1,1,-1,1]
                  ])
    b = np.zeros([8,8],dtype=int)
    for i in range(8):
        for j in range(8):
            if a[i][j] ==1:
                b[i][j]=-1
            elif a[i][j]==-1:
                b[i][j]=1
    RSS_strategy_book = np.vstack((a,b))
            
    return RSS_strategy_book

'''
def RSS(m):
    P = 2**m
    origin = FSS(m)
    RSS_sb = np.array([origin[0]])
    RSS_sb[0] = origin[0]
    def if_inco(a,b):
        eq_n = 0
        for i in range(len(a)):
            if a[i] == b[i]:
                eq_n += 1
            else:
                pass
        if eq_n == int(len(a)/2):
            return True
        else:
            return False

    for s_ori in origin:
        judge = True
        for s_cho in RSS_sb:
            if if_inco(s_ori, s_cho):
                judge = judge*True
            else:
                judge = judge*False
        if judge:
            RSS_sb = np.vstack((RSS_sb,s_ori))
            if len(RSS_sb) == P:
                break
        else:
            pass
    anti_RSS_sb = RSS_sb*(-1)
    RSS_sb = np.vstack((RSS_sb,anti_RSS_sb))
    return RSS_sb
     


In [14]:
#D(t):
def D(strategy):
    '''
    strategy is a list with length N and conponents -1, 1 or 0.
    '''
    D_t = sum(strategy)
    return D_t
    



#X(t) from t-1 step:
def lnX(lnx_tm1, strategy):
    _lambda_ = np.count_nonzero(strategy)*k0
    lnx_t = lnx_tm1 +D(strategy)/_lambda_
    return lnx_t

In [15]:
#mu(t):
def mu(mu_tm1, strategy, L, P):
    def H(x):
        return 1 if x >= 0 else 0
    _lambda_ = np.count_nonzero(strategy)*k0
    mu = 2*mu_tm1 - P*H(mu_tm1-P*0.5)+H(D(strategy)/_lambda_-L)
    return mu

#S(t):
def S(T,S_tm1, a, strategy, L):
    _lambda_ = np.count_nonzero(strategy)*k0
    if _lambda_ ==0:
        S = (1-1/T)*S_tm1
    else:
        S = (1-1/T)*S_tm1 - a*(D(strategy)/_lambda_-L)
    sigma = np.sqrt(1/(1-(1-1/T)**2))
    S_random = np.random.normal(S, sigma)
    return S_random

In [16]:
#strategy books chosen by agents:
#choose randomly and unchanged:
def choose_sb(strategy_books, N, s):
    import random
    n_sb = np.shape(strategy_books)[0]
    index_sb = set(np.arange(n_sb))
    strategy_hold = np.zeros([N,s], dtype=int)
    for i in range(N):
        strategy_hold[i] = np.array(random.sample(index_sb, s))
    return strategy_hold
    

# Steps to participate in the buy/sell game for evey agent( I am an agent!)
- I am at t now before I need to decide to buy or sell! What can I see: mu[t], decisions that other agents did at t-1.
- I check my strategy books, s strategy books are used. I don't want to change them.
- I see the mu[t], and strategy books give me s behaviors that I can act at t, or I can chooce to skip over. How should I decide?
- I see the S[t] of each of my strategy books, which can be calculated by S[t-1] and decisions that other agents did at t-1. 
- I see the max S[t], and see if it is more than r? If so, I will choose the related strategy book, and obey the behavior it offers me. If not, I won't participate in!
- Every agent like me will do the same steps, then there will be a decision list done by everyone at t. New X[t+1], mu[t+1]...
- I go to the t+1, and everything continue...

In [17]:
def buy_sell_game(t_end):
    global N, m, s, r, T, L
    index_N = np.arange(N)
    t_range = np.arange(0,t_end)
    #strategy_books = FSS(m)
    strategy_books = RSS(m)
    strategy_hold = choose_sb(strategy_books, N, s)
    lnX_list = np.zeros(t_end)
    lnX_list[0] = 1
    mu_list = np.zeros(t_end, dtype = int)
    mu_list[0] = 0
    Score = np.zeros([N,s], dtype=float)#初始化S
    strategy = np.zeros([N,t_end], dtype=int)#初始化strategy
    for t in t_range:
        if t==0:
            for n in index_N:
                for index_s in range(s):
                    strategy_index = strategy_hold[n][index_s]
                    strategy_s = strategy_books[strategy_index]
                    Score[n][index_s] = S(T,Score[n][index_s], strategy_s[mu_list[t]], [], L)
                Score_max_index = np.argmax(Score[n])
                Score_max = Score[n][Score_max_index]
                if Score_max>=r:
                    strategy_index_chosen = strategy_hold[n][Score_max_index]#选择最大分数的策略
                    strategy_s_chosen = strategy_books[strategy_index_chosen][mu_list[t]]
                    strategy[n,0] = strategy_s_chosen
                else:
                    strategy[n,0] = 0#不参加
        else:
            lnX_list[t] = lnX(lnX_list[t-1], strategy[:,t-1])
            mu_list[t] = mu(mu_list[t-1], strategy[:,t-1], L, 2**m)
            for n in index_N:
                for index_s in range(s):
                    strategy_index = strategy_hold[n][index_s]
                    strategy_s = strategy_books[strategy_index]
                    Score[n][index_s] = S(T,Score[n][index_s], strategy_s[mu_list[t]], strategy[:,t-1], L)
                Score_max_index = np.argmax(Score[n])
                Score_max = Score[n][Score_max_index]
                if Score_max>=r:
                    strategy_index_chosen = strategy_hold[n][Score_max_index]#选择最大分数的策略
                    strategy_s_chosen = strategy_books[strategy_index_chosen][mu_list[t]]
                    strategy[n,t] = strategy_s_chosen
                else:
                    strategy[n,t] = 0#不参加
    #plot figures
    t_range = np.arange(0,step)
    def plot_x_list(lnX_list):
        plt.rcParams['figure.figsize'] = (20.0, 6.0)
        plt.plot(t_range, np.exp(lnX_list))
        plt.xlabel('t')
        plt.ylabel('X')
        plt.title('X-t')
    
    def plot_Dx(strategy):
        plt.rcParams['figure.figsize'] = (20.0, 6.0)
        plt.plot(t_range,sum(strategy),lw = 0.5)
        plt.xlabel('t')
        plt.ylabel('V')
        plt.title('V-t')
    plot_x_list(lnX_list)
    plot_Dx(strategy)
    return lnX_list, strategy


    

# Culculate some statistical results
- N_active
- the distribution of Deltax
- the correlation between Deltax and x itself

In [18]:
# N_active
def plot_N_active(strategy):
    N_active_list = np.zeros(step)
    for t in range(step):
        strategy_i = strategy[:,t]
        N_active_list[t] = np.count_nonzero(strategy_i)
    t_range = np.arange(step)
    plt.rcParams['figure.figsize'] = (20.0, 6.0)
    plt.bar(t_range, N_active_list, align='center', alpha = 0.5)
    plt.xlabel('t')
    plt.ylabel('N_active')
    plt.title('N_active-t plot')
    plt.show()




In [19]:
# distribution
def distribution(x_list):
    delta_x_list = np.zeros(step-1)
    for i in range(step-1):
        delta_x_list[i] = x_list[i+1] - x_list[i]
    delta_x_min = min(delta_x_list)
    delta_x_max = max(delta_x_list)
    d_deltax = (delta_x_max-delta_x_min)/30
    delta_x_range = np.linspace(delta_x_min,delta_x_max, 30)
    distribute_n = np.zeros(30)
    for i in range(29):
        dx_min = delta_x_range[i]
        dx_max = delta_x_range[i+1]
        delta_x_list_copy = delta_x_list[:]
        for deltax in delta_x_list_copy:
            if (deltax>=dx_min) and (deltax<dx_max):
                distribute_n[i]+=1#落在区间的计数
                delta_x_list = np.delete(delta_x_list, np.where(delta_x_list == deltax))#删掉计数的
            else:
                pass
    distribute_n[-1] = len(delta_x_list)#剩下的
    plt.rcParams['figure.figsize'] = (8.0, 6.0)
    plt.bar(delta_x_range, distribute_n, width=d_deltax, align='center', alpha = 0.5)
    plt.xlabel('delta_x')
    plt.ylabel('p(delta_x)')
    plt.title('distribution of deltax')
    plt.show()


        


In [20]:
#stylised facts: fat tail
import scipy.stats as st
def QQ_plot(data_list):
    # 计算标准正态分位数，分别为0, 1/10, 2/20,..., 10/10
    quantiles = st.norm.ppf(np.linspace(0.01, 0.99, len(data_list)))
    # 对原始数据排序
    data_sort = np.sort(data_list)
    plt.scatter(quantiles, data_sort, s = 1, color='r')
    plt.xlabel('Quantiles of normal distribution') # 设置横坐标名称

    plt.ylabel('Quantiles of data') # 设置纵坐标名称

    # 绘制理论分位数直线

    line = quantiles * np.std(data_sort) + np.mean(data_sort) # 计算理论分位数直线的纵坐标值

    plt.plot(quantiles, line, 'b--') # 根据标准正态分位数和理论分位数直线的纵坐标值绘制直线，颜色为红色虚线

    plt.show() # 显示图形

In [22]:
#correlation function

def correlation_function(a_list,b_list,a_name,b_name):

    cor_list = np.array([])
    for i in np.arange(int(step/10)):
        a_list_bin = a_list[10*i:(10*i+9)]
        b_list_bin = b_list[10*i:(10*i+9)]
        cor = np.correlate(a_list_bin, b_list_bin)
        cor_list = np.append(cor_list, cor)
    step_list = np.arange(0, step, 10)
    plt.rcParams['figure.figsize'] = (8.0, 6.0)
    plt.scatter(step_list, cor_list, s = 5, marker= 'o', color = 'r')
    plt.xlabel('t')
    plt.title('correlation function of {} and {}'.format(a_name,b_name ))
    plt.ylabel('cor')


# Advanced simulation consideration
- MM model(4.10 as the score function)
- agent wealth
- time scale

In [1]:
# new S in MM
def S_MM(T, t, strategy, mu_list, strategy_book, L):
    sum_g = 0
    for i in range(t):
        mu_i = mu_list[i]
        a_i = strategy_book[mu_i]
        D_i = D(strategy[:,i])
        _lambda_ = np.count_nonzero(strategy[:,i])*k0
        part1 = (1-1/T)**(t-1-i)
        part2 = a_i*(D_i/_lambda_-L)
        sum_g += part1*part2
    S = np.sgn(sum_g)
    return S

    

In [ ]:
# consider the agent's wealth and use MM:
def buy_sell_game_wealth():
    global N, m, s, T, L, step
    index_N = np.arange(N)
    t_range = np.arange(0,step)
    #strategy_books = FSS(m)
    strategy_books = RSS(m)
    strategy_hold = choose_sb(strategy_books, N, s)
    lnX_list = np.zeros(t_end)
    lnX_list[0] = 1
    mu_list = np.zeros(step, dtype = int)
    mu_list[0] = 0
    Score = np.zeros([N,s], dtype=float)#初始化S
    strategy = np.zeros([N,step], dtype=int)#初始化strategy
    C_array = np.zeros([N,step])#初始化C
    phi_array = np.zeros([N,step])#初始化phi
    wealth_array = np.zeros([N,step])#初始化W
    for t in t_range:
        if t==0:
            x_t = np.exp(lnX_list[t])
            for n in index_N:
                phi_n = 2
                C_n = np.exp(2)
                for index_s in range(s):
                    strategy_index = strategy_hold[n][index_s]
                    strategy_s = strategy_books[strategy_index]
                    Score[n][index_s] = S_MM(T,Score[n][index_s], strategy_s[mu_list[t]], L)
                Score_max_index = np.argmax(Score[n])
                strategy_index_chosen = strategy_hold[n][Score_max_index]#选择最大分数的策略
                strategy_s_chosen = strategy_books[strategy_index_chosen][mu_list[t]]
                if ((strategy_s_chosen==-1) and (phi_n<=0)) or ((strategy_s_chosen==1)and(C_n<x_t)):
                    strategy[n][t] = 0
                else:
                    strategy[n][t] = strategy_s_chosen
                phi_array[n,t] = phi_n + strategy_s_chosen
                C_array[n,t] = C_n + x_t*strategy_s_chosen
                wealth_array[n, t] = C_array[n,t]+phi_array[n,t]*x_t

        else:
            lnX_list[t] = lnX(lnX_list[t-1], strategy[:,t-1])
            mu_list[t] = mu(mu_list[t-1], strategy[:,t-1], L, 2**m)
            x_t = np.exp(lnX_list[t])
            for n in index_N:
                phi_n = phi_array[n,t-1]
                C_n = C_array[n,t-1]
                for index_s in range(s):
                    strategy_index = strategy_hold[n][index_s]
                    strategy_s = strategy_books[strategy_index]
                    Score[n][index_s] = S_MM(T,Score[n][index_s], strategy_s[mu_list[t]], strategy, L)
                Score_max_index = np.argmax(Score[n])
                strategy_index_chosen = strategy_hold[n][Score_max_index]#选择最大分数的策略
                strategy_s_chosen = strategy_books[strategy_index_chosen][mu_list[t]]
                if ((strategy_s_chosen==-1) and (phi_n<=0)) or ((strategy_s_chosen==1)and(C_n<x_t)):
                    strategy[n][t] = 0
                else:
                    strategy[n][t] = strategy_s_chosen
                phi_array[n,t] = phi_n + strategy_s_chosen
                C_array[n,t] = C_n + x_t*strategy_s_chosen
                wealth_array[n, t] = C_array[n,t]+phi_array[n,t]*x_t
                
    #plot figures
    t_range = np.arange(0,step)
    def plot_x_list(lnX_list):
        plt.rcParams['figure.figsize'] = (20.0, 6.0)
        plt.plot(t_range, np.exp(lnX_list))
        plt.xlabel('t')
        plt.ylabel('X')
        plt.title('X-t')
    
    def plot_Dx(strategy):
        plt.rcParams['figure.figsize'] = (20.0, 6.0)
        plt.plot(t_range,sum(strategy),lw = 0.5)
        plt.xlabel('t')
        plt.ylabel('V')
        plt.title('V-t')
    plot_x_list(lnX_list)
    plot_Dx(strategy)
    return lnX_list, strategy


    

In [ ]:
#consider time scale:
def mu(mu_tm1, strategy, L, P, tau):
    def H(x):
        return 1 if x >= 0 else 0
    _lambda_ = np.count_nonzero(strategy)*k0
    mu = 2*mu_tm1 - P*H(mu_tm1-P*0.5)+H(D(strategy)/_lambda_-L)
    return mu